In [3]:
from google.colab import auth
auth.authenticate_user()


In [4]:
project_id = 'amiable-anagram-462723-j1'
bucket_name = 'musaib-sales-bucket'
file_name = 'Sample - Superstore.csv'

bq_dataset = 'sales_analytics'
bq_table = 'region_sales'


In [5]:
from google.cloud import storage
import pandas as pd
import io

client = storage.Client(project=project_id)
bucket = client.bucket(bucket_name)
blob = bucket.blob(file_name)
data_bytes = blob.download_as_bytes()
df = pd.read_csv(io.BytesIO(data_bytes), encoding='latin1')

df.head()


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [6]:
df = df.dropna(subset=['Sales'])

region_sales = df.groupby('Region', as_index=False)['Sales'].sum()

region_sales


,Region,Sales
0,Central,501239.8908
1,East,678781.2400
2,South,391721.9050
3,West,725457.8245


In [7]:
from google.cloud import bigquery

bq_client = bigquery.Client(project=project_id)
dataset_ref = bq_client.dataset(bq_dataset)
try:
    bq_client.get_dataset(dataset_ref)
except Exception:
    bq_client.create_dataset(bigquery.Dataset(dataset_ref))

table_ref = dataset_ref.table(bq_table)
job = bq_client.load_table_from_dataframe(region_sales, table_ref)
job.result()
print("Data uploaded to BigQuery!")

Data uploaded to BigQuery!
